In [8]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.feature_selection import SelectFromModel
from numpy import sort
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from xgboost.core import Booster
from matplotlib import pyplot
from xgboost import plot_importance
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import medical_lib as ml
import pandas as pd
from statistics import mean, mode
import numpy as np
import math
import warnings
import sys

from numpy.lib.function_base import average
from sklearn.utils import multiclass
from xgboost.sklearn import XGBRFClassifier
warnings.filterwarnings("ignore")
#################################################################################################

Daten einlesen

In [9]:
intervalle = [(-520, -200),(-199, 0),(1, 14),(15, 30),(31, 60),(61,90),(91,120),(121,180),(181,365),(366,850),(851,1650)]
print(intervalle)
medDatamodel2 = pd.read_csv(
    'model2_Classificationtable_intervalstatus.csv') #model2_Classificationtable_intervalstatus_TMP
medDataCopy_model2 = medDatamodel2.copy()
medDataCopy_model2 = medDataCopy_model2.iloc[:, 3:]
medDataCopy_model2_Features_Selected = medDataCopy_model2.copy()

med_class_model2 = medDataCopy_model2.iloc[:, -1]
med_features_model2 = medDataCopy_model2.iloc[:, :-1]

result = pd.read_csv('automated_algorithmen.csv')
result = result.iloc[:, 1:]

[(-520, -200), (-199, 0), (1, 14), (15, 30), (31, 60), (61, 90), (91, 120), (121, 180), (181, 365), (366, 850), (851, 1650)]


Aufteilen der Daten

In [10]:
med_features_train_model2, med_features_test_model2, med_class_train_model2, med_class_test_model2 = train_test_split(med_features_model2, med_class_model2, test_size=0.2, random_state=43, stratify=med_class_model2)
med_class_test_array = np.array(med_class_test_model2)

KNN: Training

In [11]:
medKNN = KNeighborsClassifier(n_neighbors=4)
medKNN.fit(med_features_train_model2, med_class_train_model2)

KNeighborsClassifier(n_neighbors=4)

KNN: Prediction

In [12]:
knnYpred = medKNN.predict(med_features_test_model2)
accuracyKNN = accuracy_score(knnYpred, med_class_test_array)
precisionKNN = precision_score(knnYpred, med_class_test_array, average='weighted')
recallKNN = recall_score(knnYpred, med_class_test_array, average='weighted')
f1scoreKNN = f1_score(knnYpred, med_class_test_array, average='weighted')
print('K: ', 4, 'KNN Accuracy: ', accuracyKNN, 'KNN Precision: ', precisionKNN, 'KNN Recall: ', recallKNN, 'KNN F1-Score: ', f1scoreKNN)

K:  4 KNN Accuracy:  0.7238095238095238 KNN Precision:  0.8263531335928216 KNN Recall:  0.7238095238095238 KNN F1-Score:  0.7698329592984453


KNN: Tatsächlich

In [13]:
pred_tot_lebendigknn = []
actual_tot_lebendigknn = []
abweichungknn = []
for el in range(0, len(knnYpred)):
    dist = abs(knnYpred[el] - med_class_test_array[el])
    abweichungknn.append(dist)
    if knnYpred[el] < 12:
        pred_tot_lebendigknn.append(1)
    else: 
        pred_tot_lebendigknn.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigknn.append(1)
    else:
        actual_tot_lebendigknn.append(0)
accuracyknn, precisionknn, recallknn, f1scoreknn = ml.scoring(pred_tot_lebendigknn, actual_tot_lebendigknn)
print('Tatsächlich: ', accuracyknn, precisionknn, recallknn, f1scoreknn)
print('Durchschnittliche Abweichung: ', np.mean(abweichungknn))
print('Standartabweichung der Abweichung: ', np.std(abweichungknn))

Tatsächlich:  0.7464285714285714 0.3563218390804598 0.16489361702127658 0.22545454545454544
Durchschnittliche Abweichung:  0.7261904761904762
Standartabweichung der Abweichung:  1.4078960195320034


KNN: Übertragen der Ergebnisse für Precision, Recall und F1-Score

In [14]:

index = result.index[result['Modellierung'] == 'Model_2'].tolist()
result.at[index, 'KNN_precision'] = precisionknn
result.at[index, 'KNN_recall'] = recallknn
result.at[index, 'KNN_f1'] = f1scoreknn
result.to_csv('automated_algorithmen.csv')
# pyplot.hist(abweichungknn)
# pyplot.title('Häugifkeitsverteilung der Abweichungen: Logistic Regression')
# pyplot.xlabel("Wert")
# pyplot.ylabel("Häufigkeit")
# pyplot.show()

Logistic Regression: Training

In [15]:
lr_model = LogisticRegression(solver='newton-cg', multi_class='multinomial')
lr_model.fit(med_features_train_model2, med_class_train_model2)

LogisticRegression(multi_class='multinomial', solver='newton-cg')

Logistic Regression: Prediction

In [16]:
lr_y_pred = lr_model.predict(med_features_test_model2)
lr_accuracyLogReg = accuracy_score(lr_y_pred, med_class_test_array)
lr_precisionLogReg = precision_score(
    lr_y_pred, med_class_test_array, average='weighted')
lr_recallLogReg = recall_score(
    lr_y_pred, med_class_test_array, average='weighted')
lr_f1scoreLogReg = f1_score(lr_y_pred, med_class_test_array, average='weighted')
print('Log-Regression Accuracy: ', lr_accuracyLogReg, 'Log-Regression Precision: ', lr_precisionLogReg,
      'Log-Regression Recall: ', lr_recallLogReg, 'Log-Regression F1-Score: ', lr_f1scoreLogReg)

Log-Regression Accuracy:  0.7690476190476191 Log-Regression Precision:  0.9470972954156565 Log-Regression Recall:  0.7690476190476191 Log-Regression F1-Score:  0.846468708461876


Logistic Regression: Tatsächlich

In [17]:
pred_tot_lebendiglr = []
actual_tot_lebendiglr = []
abweichunglr = []
for el in range(0, len(lr_y_pred)):
    dist = abs(lr_y_pred[el] - med_class_test_array[el])
    abweichunglr.append(dist)
    if lr_y_pred[el] < 12:
        pred_tot_lebendiglr.append(1)
    else: 
        pred_tot_lebendiglr.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendiglr.append(1)
    else:
        actual_tot_lebendiglr.append(0)
accuracylr, precisionlr, recalllr, f1scorelr = ml.scoring(pred_tot_lebendiglr, actual_tot_lebendiglr)
print('Tatsächlich: ', accuracylr, precisionlr, recalllr, f1scorelr)
print('Durchschnittliche Abweichung: ', np.mean(abweichunglr))

Tatsächlich:  0.7845238095238095 0.6129032258064516 0.10106382978723404 0.1735159817351598
Durchschnittliche Abweichung:  0.6333333333333333


Logistic Regression: Übertragen der Ergebnisse Für Precision, Recall und F1-Score

In [18]:
index = result.index[result['Modellierung'] == 'Model_2'].tolist()
result.at[index, 'Logistic_Regression_precision'] = precisionlr
result.at[index, 'Logistic_Regression_recall'] = recalllr
result.at[index, 'Logistic_Regression_f1'] = f1scorelr
result.to_csv('automated_algorithmen.csv')
# pyplot.hist(abweichunglr)
# pyplot.title('Häugifkeitsverteilung der Abweichungen: Logistic Regression')
# pyplot.xlabel("Wert")
# pyplot.ylabel("Häufigkeit")
# pyplot.show()

Decision Tree: Training

In [19]:
medical_DecTree = DecisionTreeClassifier(random_state=17)
medical_DecTree = medical_DecTree.fit(med_features_train_model2, med_class_train_model2)

Decision Tree: Prediction

In [20]:
decTree_pred = medical_DecTree.predict(med_features_test_model2)
accuracyDecTree = accuracy_score(decTree_pred, med_class_test_array)
precisionDecTree = precision_score(
    decTree_pred, med_class_test_array, average='weighted')
recallDecTree = recall_score(
    decTree_pred, med_class_test_array, average='weighted')
f1scoreDecTree = f1_score(decTree_pred, med_class_test_array, average='weighted')
print('Decision Tree: ', 'medical_DecTree Accuracy: ', accuracyDecTree, 'DecTree Precision: ',
      precisionDecTree, 'DecTree Recall: ', recallDecTree, 'DecTree F1-Score: ', f1scoreDecTree)

Decision Tree:  medical_DecTree Accuracy:  0.6583333333333333 DecTree Precision:  0.6597324005359719 DecTree Recall:  0.6583333333333333 DecTree F1-Score:  0.6589031578813412


Decision Tree: Tatsächlich

In [21]:
pred_tot_lebendigdc = []
actual_tot_lebendigdc = []
abweichungdc = []
for el in range(0, len(decTree_pred)):
    dist = abs(decTree_pred[el] - med_class_test_array[el])
    abweichungdc.append(dist)
    if decTree_pred[el] < 12:
        pred_tot_lebendigdc.append(1)
    else: 
        pred_tot_lebendigdc.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigdc.append(1)
    else:
        actual_tot_lebendigdc.append(0)
accuracydc, precisiondc, recalldc, f1scoredc = ml.scoring(pred_tot_lebendigdc, actual_tot_lebendigdc)
print('Tatsächlich: ', accuracydc, precisiondc, recalldc, f1scoredc)
print('Durchschnittliche Abweichung: ', np.mean(abweichungdc))
print('Standartabweichung der Abweichung: ', np.std(abweichungdc))

Tatsächlich:  0.7142857142857143 0.3617021276595745 0.3617021276595745 0.3617021276595745
Durchschnittliche Abweichung:  0.8654761904761905
Standartabweichung der Abweichung:  1.4325297343892367


Decision Tree: Übertragen der Ergebnisse

In [22]:
index = result.index[result['Modellierung'] == 'Model_2'].tolist()
result.at[index, 'Decision_Tree_precision'] = precisiondc
result.at[index, 'Decision_Tree_recall'] = recalldc
result.at[index, 'Decision_Tree_f1'] = f1scoredc
result.to_csv('automated_algorithmen.csv')
# pyplot.hist(abweichungdc)
# pyplot.title('Häugifkeitsverteilung der Abweichungen: Decision Tree')
# pyplot.xlabel("Wert")
# pyplot.ylabel("Häufigkeit")
# pyplot.show()

Random Forest: Training

In [23]:
medical_RF = RandomForestClassifier(
    n_estimators=100, random_state= 20)
medical_RF.fit(med_features_train_model2, med_class_train_model2)

RandomForestClassifier(random_state=20)

Random Forest: Prediction

In [24]:
rfPred = medical_RF.predict(med_features_test_model2)
accuracyRF = accuracy_score(rfPred, med_class_test_array)
precisionRF = precision_score(rfPred, med_class_test_array, average='weighted')
recallRF = recall_score(rfPred, med_class_test_array, average='weighted')
f1scoreRF = f1_score(rfPred, med_class_test_array, average='weighted')
print('Anzahl Estimator: 100 ', 'RF Accuracy: ', accuracyRF, 'RF Precision: ',
      precisionRF, 'RF Recall: ', recallRF, 'RF F1-Score: ', f1scoreRF)

Anzahl Estimator: 100  RF Accuracy:  0.7785714285714286 RF Precision:  0.9892308389706506 RF Recall:  0.7785714285714286 RF F1-Score:  0.8695010875167587


Random Forest: Tatsächlich

In [25]:
pred_tot_lebendigrf = []
actual_tot_lebendigrf = []
abweichungrf = []
for el in range(0, len(rfPred)):
    dist = abs(rfPred[el] - med_class_test_array[el])
    abweichungrf.append(dist)
    if rfPred[el] < 12:
        pred_tot_lebendigrf.append(1)
    else: 
        pred_tot_lebendigrf.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigrf.append(1)
    else:
        actual_tot_lebendigrf.append(0)
accuracyrf, precisionrf, recallrf, f1scorerf = ml.scoring(pred_tot_lebendigrf, actual_tot_lebendigrf)
print('Tatsächlich: ', accuracyrf, precisionrf, recallrf, f1scorerf)
print('Durchschnittliche Abweichung: ', np.mean(abweichungrf))
print('Standartabweichung der Abweichung: ', np.std(abweichungrf))

Tatsächlich:  0.7833333333333333 0.875 0.03723404255319149 0.07142857142857141
Durchschnittliche Abweichung:  0.6035714285714285
Standartabweichung der Abweichung:  1.3233207173956112


Random Forest: Übertragen der Ergebnisse

In [26]:
index = result.index[result['Modellierung'] == 'Model_2'].tolist()
result.at[index, 'Random_Forest_precision'] = precisionrf
result.at[index, 'Random_Forest_recall'] = recallrf
result.at[index, 'Random_Forest_f1'] = f1scorerf
result.to_csv('automated_algorithmen.csv')
# pyplot.hist(abweichungrf)
# pyplot.title('Häugifkeitsverteilung der Abweichungen: Random Forest')
# pyplot.xlabel("Wert")
# pyplot.ylabel("Häufigkeit")
# pyplot.show()

ADABoost: Training

In [27]:
adamodel = AdaBoostClassifier()
adamodel.fit(med_features_train_model2, med_class_train_model2)

AdaBoostClassifier()

ADABoost: Prediction

In [28]:
adamodel_prediction = adamodel.predict(med_features_test_model2)
adamodel_accuracy = accuracy_score(med_class_test_model2, adamodel_prediction)
adamodel_precision = precision_score(
    med_class_test_model2, adamodel_prediction, average='weighted')
adamodel_recall = recall_score(
    med_class_test_model2, adamodel_prediction, average='weighted')
adamodel_f1 = f1_score(med_class_test_model2,
                       adamodel_prediction, average='weighted')
print('ADABOOST: ', 'Accuracy: ', adamodel_accuracy, 'Precision: ',
      adamodel_precision, 'Recall: ', adamodel_recall, 'f1-Score: ', adamodel_f1)

ADABOOST:  Accuracy:  0.7761904761904762 Precision:  0.6024716553287982 Recall:  0.7761904761904762 f1-Score:  0.6783863143112473


ADABoost: Tatsächlich

In [29]:
pred_tot_lebendigada = []
actual_tot_lebendigada = []
abweichungada = []
for el in range(0, len(adamodel_prediction)):
    dist = abs(adamodel_prediction[el] - med_class_test_array[el])
    abweichungada.append(dist)
    if adamodel_prediction[el] < 12:
        pred_tot_lebendigada.append(1)
    else: 
        pred_tot_lebendigada.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigada.append(1)
    else:
        actual_tot_lebendigada.append(0)
try:
    accuracyada, precisionada, recallada, f1scoreada = ml.scoring(pred_tot_lebendigada, actual_tot_lebendigada)
    print('Tatsächlich: ', accuracyada, precisionada, recallada, f1scoreada)
    print('Durchschnittliche Abweichung: ', np.mean(abweichungada))
    print('Standartabweichung der Abweichung: ', np.std(abweichungada))
except:
    precisionada = 0.0
    recallada = 0.0
    f1scoreada = 0.0
    print('Division by Zero')

precision: Division by Zero
f1score: Division by Zero
Division by Zero


ADABoost: Übertragen der Ergebnisse

In [30]:
index = result.index[result['Modellierung'] == 'Model_2'].tolist()
result.at[index, 'ADABoost_precision'] = precisionada
result.at[index, 'ADABoost_recall'] = recallada
result.at[index, 'ADABoost_f1'] = f1scoreada
result.to_csv('automated_algorithmen.csv')
# pyplot.hist(abweichungada)
# pyplot.title('Häugifkeitsverteilung der Abweichungen: ADABoost')
# pyplot.xlabel("Wert")
# pyplot.ylabel("Häufigkeit")
# pyplot.show()

XGBoost: Training

In [31]:
xgmodel = XGBClassifier(eval_metric='mlogloss')
xgmodel.fit(med_features_train_model2, med_class_train_model2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

XGBoost: Prediction

In [32]:
xgboosted_prediction = xgmodel.predict(med_features_test_model2)
xgboosted_accuracy = accuracy_score(
    med_class_test_model2, xgboosted_prediction)
xgboosted_precision = precision_score(
    med_class_test_model2, xgboosted_prediction, average='weighted')
xgboosted_recall = recall_score(
    med_class_test_model2, xgboosted_prediction, average='weighted')
xgboosted_f1 = f1_score(med_class_test_model2,
                        xgboosted_prediction, average='weighted')
print('XGBOOST: ', 'Accuracy: ', xgboosted_accuracy, 'Precision: ',
      xgboosted_precision, 'Recall: ', xgboosted_recall, 'F1-Score: ', xgboosted_f1)

XGBOOST:  Accuracy:  0.780952380952381 Precision:  0.7049249567813924 Recall:  0.780952380952381 F1-Score:  0.7063880099285258


XGBoost: Tatsächlich

In [33]:
pred_tot_lebendigxg = []
actual_tot_lebendigxg = []
abweichungxg = []
for el in range(0, len(xgboosted_prediction)):
    dist = abs(xgboosted_prediction[el] - med_class_test_array[el])
    abweichungxg.append(dist)
    if xgboosted_prediction[el] < 12:
        pred_tot_lebendigxg.append(1)
    else: 
        pred_tot_lebendigxg.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigxg.append(1)
    else:
        actual_tot_lebendigxg.append(0)

accuracyxg, precisionxg, recallxg, f1scorexg = ml.scoring(pred_tot_lebendigxg, actual_tot_lebendigxg)
print('Tatsächlich: ', accuracyxg, precisionxg, recallxg, f1scorexg)
print('Durchschnittliche Abweichung: ', np.mean(abweichungxg))
print('Standartabweichung der Abweichung: ', np.std(abweichungxg))

Tatsächlich:  0.7976190476190477 0.78125 0.13297872340425532 0.22727272727272727
Durchschnittliche Abweichung:  0.5857142857142857
Standartabweichung der Abweichung:  1.2881458488428847


XGBoost: Übertragen der Ergebnisse

In [34]:
index = result.index[result['Modellierung'] == 'Model_2'].tolist()
result.at[index, 'XGBoost_precision'] = precisionxg
result.at[index, 'XGBoost_recall'] = recallxg
result.at[index, 'XGBoost_f1'] = f1scorexg
result.to_csv('automated_algorithmen.csv')
# pyplot.hist(abweichungxg)
# pyplot.title('Häugifkeitsverteilung der Abweichungen: XGBoost')
# pyplot.xlabel("Wert")
# pyplot.ylabel("Häufigkeit")
# pyplot.show()

XGBoost: Feature-Selection

In [35]:
featureranking = sorted((value, key) for (key, value) in xgmodel.get_booster().get_score(importance_type= 'gain').items())
# pyplot.rcParams['figure.figsize'] = [30,30]
# plot_importance(xgmodel.get_booster().get_score(importance_type= 'gain'))
# pyplot.show()

newfeatures = []
for i in range(len(featureranking)):
    if featureranking[i][0] < 1.0:
        newfeatures.append(featureranking[i][1])
# print(newfeatures)
for el in newfeatures:
    medDataCopy_model2.drop(el, inplace=True, axis=1)
medDataCopy_model2.to_csv('medDataCopy_model2_Features_Selected.csv')


Nach Feature-Selection: Einlesen der Daten

In [36]:
intervalle = [(-520, -200),(-199, 0),(1, 14),(15, 30),(31, 60),(61,90),(91,120),(121,180),(181,365),(366,850),(851,1650)]
print(intervalle)
medDatamodel2 = pd.read_csv(
    'medDataCopy_model2_Features_Selected.csv') #model2_Classificationtable_intervalstatus_TMP
medDataCopy_model2 = medDatamodel2.copy()
medDataCopy_model2 = medDataCopy_model2.iloc[:, 1:]
medDataCopy_model2_Features_Selected = medDataCopy_model2.copy()

med_class_model2 = medDataCopy_model2.iloc[:, -1]
med_features_model2 = medDataCopy_model2.iloc[:, :-1]
med_features_train_model2, med_features_test_model2, med_class_train_model2, med_class_test_model2 = train_test_split(med_features_model2, med_class_model2, test_size=0.2, random_state=43, stratify=med_class_model2)
med_class_test_array = np.array(med_class_test_model2)

result = pd.read_csv('automated_algorithmen.csv')
result = result.iloc[:, 1:]

[(-520, -200), (-199, 0), (1, 14), (15, 30), (31, 60), (61, 90), (91, 120), (121, 180), (181, 365), (366, 850), (851, 1650)]


KNN:

In [37]:
medKNN = KNeighborsClassifier(n_neighbors=4)
medKNN.fit(med_features_train_model2, med_class_train_model2)
knnYpred = medKNN.predict(med_features_test_model2)
pred_tot_lebendigknn = []
actual_tot_lebendigknn = []
abweichungknn = []
for el in range(0, len(knnYpred)):
    dist = abs(knnYpred[el] - med_class_test_array[el])
    abweichungknn.append(dist)
    if knnYpred[el] < 12:
        pred_tot_lebendigknn.append(1)
    else: 
        pred_tot_lebendigknn.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigknn.append(1)
    else:
        actual_tot_lebendigknn.append(0)
accuracyknn, precisionknn, recallknn, f1scoreknn = ml.scoring(pred_tot_lebendigknn, actual_tot_lebendigknn)
print('Tatsächlich: ', accuracyknn, precisionknn, recallknn, f1scoreknn)
print('Durchschnittliche Abweichung: ', np.mean(abweichungknn))
print('Standartabweichung der Abweichung: ', np.std(abweichungknn))
index = result.index[result['Modellierung'] == 'Model_2_selected'].tolist()
result.at[index, 'KNN_precision'] = precisionknn
result.at[index, 'KNN_recall'] = recallknn
result.at[index, 'KNN_f1'] = f1scoreknn
result.to_csv('automated_algorithmen.csv')

Tatsächlich:  0.7488095238095238 0.3707865168539326 0.17553191489361702 0.2382671480144405
Durchschnittliche Abweichung:  0.7333333333333333
Standartabweichung der Abweichung:  1.4434856402648395


Logistic Regression

In [38]:
lr_model = LogisticRegression(solver='newton-cg', multi_class='multinomial')
lr_model.fit(med_features_train_model2, med_class_train_model2)
lr_y_pred = lr_model.predict(med_features_test_model2)
pred_tot_lebendiglr = []
actual_tot_lebendiglr = []
abweichunglr = []
for el in range(0, len(lr_y_pred)):
    dist = abs(lr_y_pred[el] - med_class_test_array[el])
    abweichunglr.append(dist)
    if lr_y_pred[el] < 12:
        pred_tot_lebendiglr.append(1)
    else: 
        pred_tot_lebendiglr.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendiglr.append(1)
    else:
        actual_tot_lebendiglr.append(0)
accuracylr, precisionlr, recalllr, f1scorelr = ml.scoring(pred_tot_lebendiglr, actual_tot_lebendiglr)
print('Tatsächlich: ', accuracylr, precisionlr, recalllr, f1scorelr)
print('Durchschnittliche Abweichung: ', np.mean(abweichunglr))
print('Standartabweichung der Abweichung: ', np.std(abweichunglr))
index = result.index[result['Modellierung'] == 'Model_2_selected'].tolist()
result.at[index, 'Logistic_Regression_precision'] = precisionlr
result.at[index, 'Logistic_Regression_recall'] = recalllr
result.at[index, 'Logistic_Regression_f1'] = f1scorelr
result.to_csv('automated_algorithmen.csv')


Tatsächlich:  0.7869047619047619 0.6666666666666666 0.09574468085106383 0.16744186046511628
Durchschnittliche Abweichung:  0.6345238095238095
Standartabweichung der Abweichung:  1.355236233810588


Decision Tree

In [39]:
medical_DecTree = DecisionTreeClassifier(random_state=17)
medical_DecTree = medical_DecTree.fit(med_features_train_model2, med_class_train_model2)
decTree_pred = medical_DecTree.predict(med_features_test_model2)
pred_tot_lebendigdc = []
actual_tot_lebendigdc = []
abweichungdc = []
for el in range(0, len(decTree_pred)):
    dist = abs(decTree_pred[el] - med_class_test_array[el])
    abweichungdc.append(dist)
    if decTree_pred[el] < 12:
        pred_tot_lebendigdc.append(1)
    else: 
        pred_tot_lebendigdc.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigdc.append(1)
    else:
        actual_tot_lebendigdc.append(0)
accuracydc, precisiondc, recalldc, f1scoredc = ml.scoring(pred_tot_lebendigdc, actual_tot_lebendigdc)
print('Tatsächlich: ', accuracydc, precisiondc, recalldc, f1scoredc)
print('Durchschnittliche Abweichung: ', np.mean(abweichungdc))
print('Standartabweichung der Abweichung: ', np.std(abweichungdc))
index = result.index[result['Modellierung'] == 'Model_2_selected'].tolist()
result.at[index, 'Decision_Tree_precision'] = precisiondc
result.at[index, 'Decision_Tree_recall'] = recalldc
result.at[index, 'Decision_Tree_f1'] = f1scoredc
result.to_csv('automated_algorithmen.csv')

Tatsächlich:  0.7047619047619048 0.34375 0.35106382978723405 0.3473684210526316
Durchschnittliche Abweichung:  0.9333333333333333
Standartabweichung der Abweichung:  1.5415926623164553


Random Forest

In [40]:
medical_RF = RandomForestClassifier(
    n_estimators=100, random_state= 20)
medical_RF.fit(med_features_train_model2, med_class_train_model2)
rfPred = medical_RF.predict(med_features_test_model2)
pred_tot_lebendigrf = []
actual_tot_lebendigrf = []
abweichungrf = []
for el in range(0, len(rfPred)):
    dist = abs(rfPred[el] - med_class_test_array[el])
    abweichungrf.append(dist)
    if rfPred[el] < 12:
        pred_tot_lebendigrf.append(1)
    else: 
        pred_tot_lebendigrf.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigrf.append(1)
    else:
        actual_tot_lebendigrf.append(0)
accuracyrf, precisionrf, recallrf, f1scorerf = ml.scoring(pred_tot_lebendigrf, actual_tot_lebendigrf)
print('Tatsächlich: ', accuracyrf, precisionrf, recallrf, f1scorerf)
print('Durchschnittliche Abweichung: ', np.mean(abweichungrf))
print('Standartabweichung der Abweichung: ', np.std(abweichungrf))
index = result.index[result['Modellierung'] == 'Model_2_selected'].tolist()
result.at[index, 'Random_Forest_precision'] = precisionrf
result.at[index, 'Random_Forest_recall'] = recallrf
result.at[index, 'Random_Forest_f1'] = f1scorerf
result.to_csv('automated_algorithmen.csv')

Tatsächlich:  0.7952380952380952 0.8636363636363636 0.10106382978723404 0.18095238095238092
Durchschnittliche Abweichung:  0.5833333333333334
Standartabweichung der Abweichung:  1.2947543739601304


ADABoost

In [41]:
adamodel = AdaBoostClassifier()
adamodel.fit(med_features_train_model2, med_class_train_model2)
adamodel_prediction = adamodel.predict(med_features_test_model2)
pred_tot_lebendigada = []
actual_tot_lebendigada = []
abweichungada = []
for el in range(0, len(adamodel_prediction)):
    dist = abs(adamodel_prediction[el] - med_class_test_array[el])
    abweichungada.append(dist)
    if adamodel_prediction[el] < 12:
        pred_tot_lebendigada.append(1)
    else: 
        pred_tot_lebendigada.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigada.append(1)
    else:
        actual_tot_lebendigada.append(0)
try:
    accuracyada, precisionada, recallada, f1scoreada = ml.scoring(pred_tot_lebendigada, actual_tot_lebendigada)
    print('Tatsächlich: ', accuracyada, precisionada, recallada, f1scoreada)
    print('Durchschnittliche Abweichung: ', np.mean(abweichungada))
    print('Standartabweichung der Abweichung: ', np.std(abweichungada))
except:
    precisionada = 0.0
    print('Division by Zero')
index = result.index[result['Modellierung'] == 'Model_2_selected'].tolist()
result.at[index, 'ADABoost_precision'] = precisionada
result.at[index, 'ADABoost_recall'] = recallada
result.at[index, 'ADABoost_f1'] = f1scoreada
result.to_csv('automated_algorithmen.csv')

precision: Division by Zero
f1score: Division by Zero
Division by Zero


XGBoost

In [42]:
xgmodel = XGBClassifier(eval_metric='mlogloss')
xgmodel.fit(med_features_train_model2, med_class_train_model2)
xgboosted_prediction = xgmodel.predict(med_features_test_model2)
pred_tot_lebendigxg = []
actual_tot_lebendigxg = []
abweichungxg = []
for el in range(0, len(xgboosted_prediction)):
    dist = abs(xgboosted_prediction[el] - med_class_test_array[el])
    abweichungxg.append(dist)
    if xgboosted_prediction[el] < 12:
        pred_tot_lebendigxg.append(1)
    else: 
        pred_tot_lebendigxg.append(0)
    if med_class_test_array[el] < 12:
        actual_tot_lebendigxg.append(1)
    else:
        actual_tot_lebendigxg.append(0)

accuracyxg, precisionxg, recallxg, f1scorexg = ml.scoring(pred_tot_lebendigxg, actual_tot_lebendigxg)
print('Tatsächlich: ', accuracyxg, precisionxg, recallxg, f1scorexg)
print('Durchschnittliche Abweichung: ', np.mean(abweichungxg))
print('Standartabweichung der Abweichung: ', np.std(abweichungxg))
index = result.index[result['Modellierung'] == 'Model_2_selected'].tolist()
result.at[index, 'XGBoost_precision'] = precisionxg
result.at[index, 'XGBoost_recall'] = recallxg
result.at[index, 'XGBoost_f1'] = f1scorexg
result.to_csv('automated_algorithmen.csv')

Tatsächlich:  0.7988095238095239 0.7021276595744681 0.17553191489361702 0.28085106382978725
Durchschnittliche Abweichung:  0.6071428571428571
Standartabweichung der Abweichung:  1.332897462769925
